## Update Take II: The Updatening!

Porting updates from prior project.

In [35]:
new_dir_jsons = '~/Downloads/tmp/JCDL2025/tmp_save/example_hists_updates/jsons/' # where to store 
new_dir_pickles = '~/Downloads/tmp/JCDL2025/tmp_save/example_hists_updates/pickles/' # where to store 

orig_dir_jsons = '~/Downloads/tmp/JCDL2025/visual_qa_histograms/example_hists/jsons/'
orig_dir_pickles = '~/Downloads/tmp/JCDL2025/visual_qa_histograms/example_hists/LLM_outputs/chatgpt_api/'

In [36]:
import json
from glob import glob
import numpy as np
from copy import deepcopy
import os
import pickle

# debug
from importlib import reload

from sys import path
path.append('../')
from utils.plot_qa_utils import init_qa_pairs

from utils.data_utils import NumpyEncoder

from utils.plot_parameters import plot_types_params
plot_params = plot_types_params.copy()

import utils.plot_qa_utils
reload(utils.plot_qa_utils)

verbose_qa = False

In [37]:
new_dir_jsons = os.path.expanduser(new_dir_jsons)
orig_dir_jsons = os.path.expanduser(orig_dir_jsons)

new_dir_pickles = os.path.expanduser(new_dir_pickles)
orig_dir_pickles = os.path.expanduser(orig_dir_pickles)

In [38]:
# stats for doing calculations
stats = {'minimum':np.min, 'maximum':np.max, 'median':np.median, 'mean':np.mean}

In [39]:
files_json_qa = glob(orig_dir_jsons + '*.json*')
files_json_qa[:3]

['/Users/jnaiman/Downloads/tmp/JCDL2025/visual_qa_histograms/example_hists/jsons/id_0077.json',
 '/Users/jnaiman/Downloads/tmp/JCDL2025/visual_qa_histograms/example_hists/jsons/id_0020.json',
 '/Users/jnaiman/Downloads/tmp/JCDL2025/visual_qa_histograms/example_hists/jsons/id_0036.json']

In [40]:
import utils.plot_qa_utils
reload(utils.plot_qa_utils)
from utils.plot_qa_utils import init_qa_pairs

import utils.histogram_plot_qa_utils
reload(utils.histogram_plot_qa_utils)

from utils.histogram_plot_qa_utils import q_nbars_hist_plot_plotnums, q_stats_hists, q_gmm_ngaussians_hists

In [41]:
for json_path in files_json_qa:
    print('**************', json_path.split('/')[-1].split('.json')[0], '*************')
    with open(json_path, 'r') as f:
        qa_orig = json.load(f)
        qa_orig = json.loads(qa_orig)

    datas = deepcopy(qa_orig)
    axes = [[]] # fake it

    ######## GENERATE QA BASE #######
    qa_pairs = init_qa_pairs()
    for iplot in range(len((axes))):
        if datas['plot'+str(iplot)]['type'] == 'histogram':
            ############ L1 #############
            # number of bars
            if len(axes) > 1:
                qa_pairs = q_nbars_hist_plot_plotnums(datas, qa_pairs, 
                                                      plot_num = iplot, 
                                                      use_words=False, 
                                                      verbose=verbose_qa)
            qa_pairs = q_nbars_hist_plot_plotnums(datas, qa_pairs, 
                                                  plot_num = iplot, 
                                                  use_words=True, 
                                                  verbose=verbose_qa)

            ###### L2 #######
            # stats items
            for k,v in stats.items():
                if len(axes) > 1:
                    qa_pairs = q_stats_hists(datas, qa_pairs, stat={k:v}, 
                                             plot_num=iplot, use_words=False, 
                                             verbose=verbose_qa)
                qa_pairs = q_stats_hists(datas, qa_pairs, stat={k:v}, 
                                         plot_num=iplot, use_words=True, 
                                         verbose=verbose_qa)

            ###### L3 ######
            # if GMM -- how many gaussians?
            hasGMM = False
            if 'data params' not in datas['plot'+str(iplot)]['data']:
                #print('Not a gmm relationship!')
                pass
            else:
                if datas['plot'+str(iplot)]['distribution'] == 'gmm':
                    hasGMM = True
            if hasGMM:       
                qa_pairs = q_gmm_ngaussians_hists(datas, qa_pairs, 
                                                  plot_num=iplot, 
                                                  use_words=True, 
                                                  verbose=verbose_qa, 
                                                  additional_format = 'Please choose an integer number from 1 to 5.')

    datas_out = deepcopy(datas)
    datas_out['VQA'] = qa_pairs
    dumped = json.dumps(datas_out, cls=NumpyEncoder)
    with open(new_dir_jsons + json_path.split('/')[-1], 'w') as f:
        json.dump(dumped, f)

    # re-read and check
    with open(new_dir_jsons + json_path.split('/')[-1], 'r') as f:
        datas_out = json.load(f)
        datas_out = json.loads(datas_out)

    # tests
    if datas_out['figure'] != datas['figure']:
        print('Figures data not equal!')
        import sys; sys.exit()
    if datas_out['plot0'] != datas['plot0']:
        print('Plots not equal!')
        import sys; sys.exit()
    for (k,v),(ko,vo) in zip(datas.items(), datas_out.items()):
        if k != ko: # shouldn't happen
            print('Different keys:', k, ko)
        if v != vo:
            print('Orig:')
            print(v)
            print('New:')
            print(vo)
            print('')

    #json_out = deepcopy(datas_out)
    #json_out['VQA'] = deepcopy(qa_pairs)
    # dump full data



    #import sys; sys.exit()



************** id_0077 *************
************** id_0020 *************
************** id_0036 *************
************** id_0061 *************
************** id_0016 *************
************** id_0041 *************
Orig:
{'Level 1': {'Figure-level questions': {}, 'Plot-level questions': {'nbars': {'plot0': {'Q': 'You are a helpful assistant that can analyze images.  How many bars are there in the specified figure panel? Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.', 'A': {'nbars': 50}, 'persona': 'You are a helpful assistant that can analyze images.', 'context': '', 'question': 'How many bars are there in the specified figure panel?', 'format': 'Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.'}}}}, 'Level 2': {'Plot-level questions': {'minimum': {'plot0': {'Q': 'You are a helpful assistant that can analyze images.  What is the minimum